In [ ]:
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
#from keras.layers.normalization import BatchNormalization
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
import random

random.seed(0)
tf.random.set_seed(0)
weight_decay = 0.0000005

In [ ]:
import abc
import itertools
import numpy as np
from keras.preprocessing.image import apply_affine_transform

## **Load and Preprocessing the data**

In [ ]:
def resize_and_crop_image(input_file, output_side_length, greyscale=False):
    img = cv2.imread(input_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB if not greyscale else cv2.COLOR_BGR2GRAY)
    height, width = img.shape[:2]
    new_height = output_side_length
    new_width = output_side_length
    if height > width:
        new_height = int(output_side_length * height / width)
    else:
        new_width = int(output_side_length * width / height)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    height_offset = (new_height - output_side_length) // 2
    width_offset = (new_width - output_side_length) // 2
    cropped_img = resized_img[height_offset:height_offset + output_side_length,
                              width_offset:width_offset + output_side_length]
    assert cropped_img.shape[:2] == (output_side_length, output_side_length)
    return cropped_img

In [ ]:
def normalize_minus1_1(data):
    return 2*(data/255.) - 1

In [ ]:
def get_channels_axis():
  import keras
  idf = keras.backend.image_data_format()
  if idf == 'channels_first':
      return 1
  assert idf == 'channels_last'
  return 3

In [ ]:
def load_fashion_mnist():
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(np.pad(X_train, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_train = np.expand_dims(X_train, axis=get_channels_axis())
    X_test = normalize_minus1_1(cast_to_floatx(np.pad(X_test, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_test = np.expand_dims(X_test, axis=get_channels_axis())
    return (X_train, y_train), (X_test, y_test)

In [ ]:
def load_cifar10():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(X_train))
    X_test = normalize_minus1_1(cast_to_floatx(X_test))
    return (X_train, y_train), (X_test, y_test)

## **Saving the results**

In [ ]:
def roc_pr_curve_data(scores, labels):
    scores = scores.flatten()
    labels = labels.flatten()

    scores_pos = scores[labels == 1]
    scores_neg = scores[labels != 1]

    truth = np.concatenate((np.zeros_like(scores_neg), np.ones_like(scores_pos)))
    preds = np.concatenate((scores_neg, scores_pos))
    fpr, tpr, roc_thresholds = roc_curve(truth, preds)
    roc_auc = auc(fpr, tpr)
    print("roc_auc",roc_auc)

    # pr curve where "normal" is the positive class
    precision_norm, recall_norm, pr_thresholds_norm = precision_recall_curve(truth, preds)
    pr_auc_norm = auc(recall_norm, precision_norm)
    print("pr_auc_norm where normal is the positive class",pr_auc_norm)

    # pr curve where "anomaly" is the positive class
    precision_anom, recall_anom, pr_thresholds_anom = precision_recall_curve(truth, -preds, pos_label=0)
    pr_auc_anom = auc(recall_anom, precision_anom)
    print("pr_auc_norm where anomaly is the positive class",pr_auc_anom)

In [ ]:
def get_class_name_from_index(index, dataset_name):
    ind_to_name = {
        'fashion-mnist': ('t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag',
                          'ankle-boot'),
    }

    return ind_to_name[dataset_name][index]

## **Transformations**

In [ ]:
class AffineTransformation(object):
    def __init__(self, flip, tx, ty, k_90_rotate):
        self.flip = flip
        self.tx = tx
        self.ty = ty
        self.k_90_rotate = k_90_rotate

    def __call__(self, x):
        res_x = x
        if self.flip:
            res_x = np.fliplr(res_x)
        if self.tx != 0 or self.ty != 0:
            res_x = apply_affine_transform(res_x, tx=self.tx, ty=self.ty,row_axis=0,
    col_axis=1, channel_axis=2, fill_mode='reflect')
        if self.k_90_rotate != 0:
            res_x = np.rot90(res_x, self.k_90_rotate)

        return res_x

In [ ]:
class AbstractTransformer(abc.ABC):
    def __init__(self):
        self._transformation_list = None
        self._create_transformation_list()

    @property
    def n_transforms(self):
        return len(self._transformation_list)

    @abc.abstractmethod
    def _create_transformation_list(self):
        return

    def transform_batch(self, x_batch, t_inds):
        assert len(x_batch) == len(t_inds)

        transformed_batch = x_batch.copy()
        for i, t_ind in enumerate(t_inds):
            transformed_batch[i] = self._transformation_list[t_ind](transformed_batch[i])
        return transformed_batch

In [ ]:
class Transformer(AbstractTransformer):
    def __init__(self, translation_x=8, translation_y=8):
        self.max_tx = translation_x
        self.max_ty = translation_y
        super().__init__()

    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, tx, ty, k_rotate in itertools.product((False, True),
                                                           (0, -self.max_tx, self.max_tx),
                                                           (0, -self.max_ty, self.max_ty),
                                                           range(2)):
            transformation = AffineTransformation(is_flip, tx, ty, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

In [ ]:
class SimpleTransformer(AbstractTransformer):
    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, k_rotate in itertools.product((False, True),
                                                    range(2)):
            transformation = AffineTransformation(is_flip, 0, 0, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

## **The Model: Wide Residual Network**

In [ ]:
def initial_conv(input):
    x = Convolution2D(16, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(input)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    return x


def expand_conv(init, base, k, strides=(1, 1)):
    x = Convolution2D(base * k, (3, 3), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = Convolution2D(base * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    skip = Convolution2D(base * k, (1, 1), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    m = Add()([x, skip])

    return m


def conv1_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv2_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv3_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    ip = Input(shape=input_dim)

    x = initial_conv(ip)
    nb_conv = 4

    x = expand_conv(x, 16, k)
    nb_conv += 2

    for i in range(N - 1):
        x = conv1_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 32, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv2_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 64, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv3_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)

    x = Dense(nb_classes, kernel_regularizer=l2(weight_decay), activation='softmax')(x)

    model = Model(ip, x)

    if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
    return model

## **Experiments**

In [ ]:
import os
import csv
from collections import defaultdict
from glob import glob
from datetime import datetime
from multiprocessing import Manager, freeze_support, Process
import numpy as np
import scipy.stats
from scipy.special import psi, polygamma, gamma
from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
from keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
from tqdm.notebook import tqdm

In [ ]:
def _transformations_experiment(dataset_load_fn, dataset_name, single_class_ind, gpu_q):
    gpu_to_use = gpu_q.get()
    print('training class:',single_class_ind)
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_to_use

    (x_train, y_train), (x_test, y_test) = dataset_load_fn()
    x_train = x_train[:TRAIN_SIZE]
    y_train =  y_train[:TRAIN_SIZE]
    x_test =  x_test[:VAL_SIZE]
    y_test =  y_test[:VAL_SIZE]

    transformer = Transformer(8, 8)
    n, k = (10, 4)
    mdl = create_wide_residual_network(x_train.shape[1:], transformer.n_transforms, n, k)
    mdl.compile('adam',
                'categorical_crossentropy',
                ['acc'])

    x_train_task = x_train[y_train.flatten() == single_class_ind]
    transformations_inds = np.tile(np.arange(transformer.n_transforms), len(x_train_task))
    print('Start of the proposed transformation')
    x_train_task_transformed = transformer.transform_batch(np.repeat(x_train_task, transformer.n_transforms, axis=0),
                                                           transformations_inds)
    print('End of the proposed transformation')
    batch_size = 128
    print('Start of the training')
    mdl.fit(x=x_train_task_transformed, y=to_categorical(transformations_inds),
            batch_size=64, epochs= 5
            )
    print('End of the training')

    #Estimating the parameters
    def inv_psi(y, iters=5):
        # initial estimate
        cond = y >= -2.22
        x = cond * (np.exp(y) + 0.5) + (1 - cond) * -1 / (y - psi(1))

        for _ in range(iters):
            x = x - (psi(x) - y) / polygamma(1, x)
        return x

    def fixed_point_beta_louiville_mle(vect_alpha_0, alpha_0, beta_0,lamda_0, y, max_iter=50):
        y = np.asarray(y)
        (N,K) = (y.shape[0], y.shape[1])
        alpha_old = alpha_0
        beta_old = beta_0
        lamda_old = lamda_0
        vect_alpha_old = vect_alpha_0

        for iterr in range(max_iter):
          aph = psi(alpha_old + beta_old) + np.log(lamda_old) + (1/N)*np.sum(np.log(np.max(y,axis=1))) - (1/N)*np.sum([np.abs(np.log(1 - (1-lamda_old)*np.max(y[j]))) for j in range(N)])
          aph = np.nan_to_num(aph, copy=True, nan=10**7, posinf=10**10, neginf=-(10**10))
          alpha_new = inv_psi(np.clip(aph,None,10), iters=5)
          alpha_new = np.clip(alpha_new,10**(-10),None)

          #calculate beta
          bet = psi(alpha_old + beta_old) + (1/N)*np.sum(np.log(np.clip(1-np.max(y,axis=1),10**(-10),None))) - (1/N)*np.sum([np.log(np.abs(1 - (1-lamda_old)*np.max(y[j]))) for j in range(N)])
          bet = np.nan_to_num(bet, copy=True, nan=10**7, posinf=10**10, neginf=-(10**10))
          beta_new = inv_psi(np.clip(bet,None,10), iters=5)
          beta_new = np.clip(beta_new,10**(-10),None)

          #calculate lamda
          lamda_new = (N*alpha_old) / np.sum([(alpha_old+beta_old)*np.max(y[j])/(1-(1-lamda_old)*np.max(y[j])) for j in range(N)])
          lamda_new = np.nan_to_num(lamda_new, copy=True, nan=10**7, posinf=10**10, neginf=-(10**10))
          lamda_new = np.clip(lamda_new,10**(-10),None)

          #calculate the vector alpha
          vect_alpha_new = np.asarray([inv_psi(psi(np.sum(vect_alpha_old)) + (1/N)*np.sum([np.log(y[j][i]) for j in range(N)]) - (1/N)*np.sum(np.max(y,axis=1)) ,iters=5) for i in range(K)])
          vect_alpha_new = np.nan_to_num((np.clip(vect_alpha_new,10**(-10),None)), copy=True, nan=10**7, posinf=10**10, neginf=-(10**10))

          #Update the parameters
          alpha_old = alpha_new
          beta_old = beta_new
          lamda_old = lamda_new
          vect_alpha_old = vect_alpha_new
        return vect_alpha_new, alpha_new, beta_new, lamda_new

    #Normality Score
    def lnbl_normality_score(vect_alpha,alpha,beta,lamda,y):
      (N,K) = (y.shape[0], y.shape[1])
      scores = np.asarray([ (alpha-np.sum(vect_alpha))*np.log(np.max(y[j])) + (beta-1)*np.log(np.clip(1-np.max(y[j]),10**(-10),None)) - (alpha+beta)*np.log(np.abs(1-(1-lamda)*np.max(y[j]))) + np.sum((vect_alpha-1)*np.log(y[j])) for j in range(N)])
      scores = np.nan_to_num(scores, copy=True, nan=10**7, posinf=10**10, neginf=-(10**10))
      return scores

    scores = np.zeros((len(x_test),))
    observed_data = x_train_task
    for t_ind in range(transformer.n_transforms):
        observed_dirichlet = mdl.predict(transformer.transform_batch(observed_data, [t_ind] * len(observed_data)),
                                         batch_size=64)
        vect_alpha_0 = 0.005*np.ones(observed_dirichlet.shape[1])
        alpha_0 = 0.1
        beta_0 = 0.1
        lamda_0 = 0.1

        mle_vect_alpha_t,mle_alpha_t,mle_beta_t,mle_lamda_t = fixed_point_beta_louiville_mle(vect_alpha_0, alpha_0, beta_0,lamda_0,observed_dirichlet,max_iter=30)
        mle_vect_alpha_t=np.nan_to_num(mle_vect_alpha_t, copy=True, nan=10**3, posinf=10**10, neginf=-10*10)
        mle_alpha_t= np.nan_to_num(mle_alpha_t, copy=True, nan=10**7, posinf=10**10, neginf=-10*10)
        mle_beta_t= np.nan_to_num(mle_beta_t, copy=True, nan=10**7, posinf=10**10, neginf=-10**10)
        mle_lamda_t= np.nan_to_num(mle_lamda_t, copy=True, nan=10**7, posinf=10**10, neginf=-10**10)

        x_test_p = mdl.predict(transformer.transform_batch(x_test, [t_ind] * len(x_test)),
                               batch_size=64)
        scores += lnbl_normality_score(mle_vect_alpha_t,mle_alpha_t,mle_beta_t,mle_lamda_t, x_test_p)
        scores = np.nan_to_num(scores, copy=True, nan=10**7, posinf=10**10, neginf=-(10**10))

    scores /= transformer.n_transforms
    labels = y_test.flatten() == single_class_ind
    roc_pr_curve_data(scores, labels)

    gpu_q.put(gpu_to_use)

In [ ]:
def run_experiments(load_dataset_fn, dataset_name, q, n_classes):
    print("START _transformations_experiment ")
    n_runs = 3
    # Transformations
    for i in range(n_runs):
        print('Run number:',i+1)
        processes = [Process(target=_transformations_experiment,
                             args=(load_dataset_fn, dataset_name, c, q)) for c in range(n_classes)]
        for p in processes:
            p.start()
        for p in processes:
            p.join()
    print("END _transformations_experiment ")

In [ ]:
RESULTS_DIR = ''
TRAIN_SIZE = 10000
VAL_SIZE = 1000
BATCH_SIZE = 64
N_EPOCHS = 5

In [ ]:
if __name__ == '__main__':
    freeze_support()
    N_GPUS = 1
    man = Manager()
    q = man.Queue(N_GPUS)
    for g in range(N_GPUS):
        q.put(str(g))

    experiments_list = [
        (load_fashion_mnist, 'fashion-mnist', 10),
        #(load_cifar10, 'cifar10', 10),
    ]

    for data_load_fn, dataset_name, n_classes in experiments_list:
        run_experiments(data_load_fn, dataset_name, q, n_classes)

START _transformations_experiment 
Run number: 1
training class: 0
Wide Residual Network-64-4 created.
Start of the proposed transformation
End of the proposed transformation
Start of the training
Epoch 1/5


204/530 [==========>...................] - ETA: 55:43 - loss: 3.5906 - acc: 0.04 - ETA: 13:08 - loss: 3.5908 - acc: 0.03 - ETA: 12:51 - loss: 3.5894 - acc: 0.07 - ETA: 13:47 - loss: 3.5861 - acc: 0.08 - ETA: 13:49 - loss: 3.5855 - acc: 0.08 - ETA: 13:29 - loss: 3.5829 - acc: 0.10 - ETA: 13:15 - loss: 3.5783 - acc: 0.11 - ETA: 13:04 - loss: 3.5734 - acc: 0.12 - ETA: 12:56 - loss: 3.5716 - acc: 0.12 - ETA: 12:50 - loss: 3.5679 - acc: 0.12 - ETA: 12:42 - loss: 3.5641 - acc: 0.12 - ETA: 12:35 - loss: 3.5591 - acc: 0.13 - ETA: 12:30 - loss: 3.5543 - acc: 0.13 - ETA: 12:26 - loss: 3.5486 - acc: 0.14 - ETA: 12:22 - loss: 3.5430 - acc: 0.15 - ETA: 12:17 - loss: 3.5374 - acc: 0.15 - ETA: 12:14 - loss: 3.5324 - acc: 0.15 - ETA: 12:09 - loss: 3.5268 - acc: 0.16 - ETA: 12:07 - loss: 3.5210 - acc: 0.16 - ETA: 12:02 - loss: 3.5154 - acc: 0.16 - ETA: 11:59 - loss: 3.5086 - acc: 0.17 - ETA: 11:56 - loss: 3.5035 - acc: 0.16 - ETA: 11:53 - loss: 3.4981 - acc: 0.17 - ETA: 11:51 - loss: 3.4919 - acc: 0.17

408/530 [======================>.......] - ETA: 7:11 - loss: 1.8153 - acc: 0.418 - ETA: 7:10 - loss: 1.8110 - acc: 0.418 - ETA: 7:09 - loss: 1.8062 - acc: 0.418 - ETA: 7:07 - loss: 1.8019 - acc: 0.419 - ETA: 7:06 - loss: 1.7970 - acc: 0.419 - ETA: 7:04 - loss: 1.7923 - acc: 0.420 - ETA: 7:03 - loss: 1.7885 - acc: 0.420 - ETA: 7:02 - loss: 1.7837 - acc: 0.421 - ETA: 7:00 - loss: 1.7797 - acc: 0.421 - ETA: 6:59 - loss: 1.7755 - acc: 0.422 - ETA: 6:58 - loss: 1.7712 - acc: 0.422 - ETA: 6:56 - loss: 1.7667 - acc: 0.423 - ETA: 6:55 - loss: 1.7626 - acc: 0.423 - ETA: 6:54 - loss: 1.7592 - acc: 0.423 - ETA: 6:52 - loss: 1.7550 - acc: 0.424 - ETA: 6:51 - loss: 1.7506 - acc: 0.424 - ETA: 6:50 - loss: 1.7466 - acc: 0.425 - ETA: 6:48 - loss: 1.7423 - acc: 0.425 - ETA: 6:47 - loss: 1.7387 - acc: 0.425 - ETA: 6:45 - loss: 1.7348 - acc: 0.425 - ETA: 6:44 - loss: 1.7315 - acc: 0.426 - ETA: 6:43 - loss: 1.7274 - acc: 0.426 - ETA: 6:42 - loss: 1.7239 - acc: 0.426 - ETA: 6:40 - loss: 1.7198 - acc: 0.426

530/530 [==============================] - ETA: 2:39 - loss: 1.3151 - acc: 0.455 - ETA: 2:37 - loss: 1.3138 - acc: 0.455 - ETA: 2:36 - loss: 1.3124 - acc: 0.455 - ETA: 2:35 - loss: 1.3111 - acc: 0.455 - ETA: 2:34 - loss: 1.3096 - acc: 0.456 - ETA: 2:32 - loss: 1.3083 - acc: 0.456 - ETA: 2:31 - loss: 1.3069 - acc: 0.456 - ETA: 2:30 - loss: 1.3056 - acc: 0.456 - ETA: 2:28 - loss: 1.3043 - acc: 0.456 - ETA: 2:27 - loss: 1.3031 - acc: 0.456 - ETA: 2:26 - loss: 1.3019 - acc: 0.456 - ETA: 2:24 - loss: 1.3007 - acc: 0.456 - ETA: 2:23 - loss: 1.2999 - acc: 0.456 - ETA: 2:22 - loss: 1.2986 - acc: 0.456 - ETA: 2:20 - loss: 1.2974 - acc: 0.456 - ETA: 2:19 - loss: 1.2962 - acc: 0.456 - ETA: 2:18 - loss: 1.2948 - acc: 0.456 - ETA: 2:16 - loss: 1.2935 - acc: 0.457 - ETA: 2:15 - loss: 1.2924 - acc: 0.457 - ETA: 2:14 - loss: 1.2912 - acc: 0.457 - ETA: 2:12 - loss: 1.2898 - acc: 0.457 - ETA: 2:11 - loss: 1.2885 - acc: 0.457 - ETA: 2:10 - loss: 1.2872 - acc: 0.457 - ETA: 2:08 - loss: 1.2861 - acc: 0.458

204/530 [==========>...................] - ETA: 11:31 - loss: 0.7045 - acc: 0.50 - ETA: 11:30 - loss: 0.7090 - acc: 0.54 - ETA: 11:24 - loss: 0.7164 - acc: 0.56 - ETA: 11:24 - loss: 0.7118 - acc: 0.57 - ETA: 11:23 - loss: 0.7199 - acc: 0.55 - ETA: 11:22 - loss: 0.7204 - acc: 0.54 - ETA: 11:25 - loss: 0.7322 - acc: 0.54 - ETA: 11:24 - loss: 0.7319 - acc: 0.53 - ETA: 11:22 - loss: 0.7335 - acc: 0.53 - ETA: 11:20 - loss: 0.7296 - acc: 0.54 - ETA: 11:20 - loss: 0.7284 - acc: 0.55 - ETA: 11:19 - loss: 0.7251 - acc: 0.55 - ETA: 11:18 - loss: 0.7279 - acc: 0.54 - ETA: 11:17 - loss: 0.7285 - acc: 0.54 - ETA: 11:15 - loss: 0.7247 - acc: 0.54 - ETA: 11:13 - loss: 0.7297 - acc: 0.53 - ETA: 11:12 - loss: 0.7297 - acc: 0.53 - ETA: 11:12 - loss: 0.7377 - acc: 0.52 - ETA: 11:10 - loss: 0.7383 - acc: 0.52 - ETA: 11:08 - loss: 0.7391 - acc: 0.52 - ETA: 11:06 - loss: 0.7395 - acc: 0.52 - ETA: 11:05 - loss: 0.7381 - acc: 0.52 - ETA: 11:04 - loss: 0.7380 - acc: 0.52 - ETA: 11:03 - loss: 0.7379 - acc: 0.52

...

408/563 [====================>.........] - ETA: 8:56 - loss: 0.0231 - acc: 0.996 - ETA: 8:55 - loss: 0.0230 - acc: 0.996 - ETA: 8:53 - loss: 0.0231 - acc: 0.996 - ETA: 8:52 - loss: 0.0230 - acc: 0.996 - ETA: 8:50 - loss: 0.0230 - acc: 0.996 - ETA: 8:49 - loss: 0.0229 - acc: 0.996 - ETA: 8:47 - loss: 0.0229 - acc: 0.996 - ETA: 8:46 - loss: 0.0228 - acc: 0.996 - ETA: 8:44 - loss: 0.0228 - acc: 0.996 - ETA: 8:43 - loss: 0.0228 - acc: 0.996 - ETA: 8:41 - loss: 0.0227 - acc: 0.996 - ETA: 8:40 - loss: 0.0227 - acc: 0.997 - ETA: 8:38 - loss: 0.0227 - acc: 0.997 - ETA: 8:37 - loss: 0.0227 - acc: 0.997 - ETA: 8:35 - loss: 0.0227 - acc: 0.997 - ETA: 8:34 - loss: 0.0227 - acc: 0.997 - ETA: 8:33 - loss: 0.0226 - acc: 0.997 - ETA: 8:31 - loss: 0.0227 - acc: 0.997 - ETA: 8:30 - loss: 0.0227 - acc: 0.997 - ETA: 8:28 - loss: 0.0226 - acc: 0.997 - ETA: 8:27 - loss: 0.0226 - acc: 0.997 - ETA: 8:25 - loss: 0.0225 - acc: 0.997 - ETA: 8:24 - loss: 0.0225 - acc: 0.997 - ETA: 8:22 - loss: 0.0224 - acc: 0.997

563/563 [==============================] - ETA: 3:50 - loss: 0.0275 - acc: 0.996 - ETA: 3:49 - loss: 0.0276 - acc: 0.996 - ETA: 3:47 - loss: 0.0275 - acc: 0.996 - ETA: 3:46 - loss: 0.0276 - acc: 0.995 - ETA: 3:44 - loss: 0.0276 - acc: 0.996 - ETA: 3:43 - loss: 0.0275 - acc: 0.996 - ETA: 3:41 - loss: 0.0275 - acc: 0.996 - ETA: 3:40 - loss: 0.0275 - acc: 0.996 - ETA: 3:38 - loss: 0.0274 - acc: 0.996 - ETA: 3:37 - loss: 0.0274 - acc: 0.996 - ETA: 3:35 - loss: 0.0274 - acc: 0.996 - ETA: 3:34 - loss: 0.0275 - acc: 0.996 - ETA: 3:32 - loss: 0.0275 - acc: 0.996 - ETA: 3:31 - loss: 0.0274 - acc: 0.996 - ETA: 3:29 - loss: 0.0274 - acc: 0.996 - ETA: 3:28 - loss: 0.0274 - acc: 0.996 - ETA: 3:26 - loss: 0.0274 - acc: 0.996 - ETA: 3:25 - loss: 0.0273 - acc: 0.996 - ETA: 3:23 - loss: 0.0273 - acc: 0.996 - ETA: 3:22 - loss: 0.0275 - acc: 0.995 - ETA: 3:20 - loss: 0.0275 - acc: 0.995 - ETA: 3:19 - loss: 0.0275 - acc: 0.995 - ETA: 3:17 - loss: 0.0275 - acc: 0.995 - ETA: 3:16 - loss: 0.0275 - acc: 0.995

204/563 [=========>....................] - ETA: 14:12 - loss: 0.0133 - acc: 1.00 - ETA: 13:35 - loss: 0.0133 - acc: 1.00 - ETA: 13:47 - loss: 0.0161 - acc: 1.00 - ETA: 13:51 - loss: 0.0300 - acc: 0.99 - ETA: 13:49 - loss: 0.0266 - acc: 0.99 - ETA: 13:44 - loss: 0.0244 - acc: 0.99 - ETA: 13:42 - loss: 0.0229 - acc: 0.99 - ETA: 13:42 - loss: 0.0216 - acc: 0.99 - ETA: 13:41 - loss: 0.0211 - acc: 0.99 - ETA: 13:41 - loss: 0.0258 - acc: 0.99 - ETA: 13:40 - loss: 0.0245 - acc: 0.99 - ETA: 13:39 - loss: 0.0245 - acc: 0.99 - ETA: 13:37 - loss: 0.0239 - acc: 0.99 - ETA: 13:36 - loss: 0.0231 - acc: 0.99 - ETA: 13:34 - loss: 0.0224 - acc: 0.99 - ETA: 13:33 - loss: 0.0230 - acc: 0.99 - ETA: 13:31 - loss: 0.0224 - acc: 0.99 - ETA: 13:31 - loss: 0.0228 - acc: 0.99 - ETA: 13:30 - loss: 0.0222 - acc: 0.99 - ETA: 13:30 - loss: 0.0217 - acc: 0.99 - ETA: 13:28 - loss: 0.0213 - acc: 0.99 - ETA: 13:26 - loss: 0.0210 - acc: 0.99 - ETA: 13:25 - loss: 0.0206 - acc: 0.99 - ETA: 13:23 - loss: 0.0204 - acc: 0.99

408/563 [====================>.........] - ETA: 8:55 - loss: 0.0152 - acc: 0.999 - ETA: 8:53 - loss: 0.0152 - acc: 0.999 - ETA: 8:52 - loss: 0.0152 - acc: 0.999 - ETA: 8:50 - loss: 0.0152 - acc: 0.999 - ETA: 8:49 - loss: 0.0151 - acc: 0.999 - ETA: 8:47 - loss: 0.0151 - acc: 0.999 - ETA: 8:46 - loss: 0.0151 - acc: 0.999 - ETA: 8:44 - loss: 0.0151 - acc: 0.999 - ETA: 8:43 - loss: 0.0151 - acc: 0.999 - ETA: 8:41 - loss: 0.0151 - acc: 0.999 - ETA: 8:40 - loss: 0.0151 - acc: 0.999 - ETA: 8:38 - loss: 0.0150 - acc: 0.999 - ETA: 8:37 - loss: 0.0150 - acc: 0.999 - ETA: 8:35 - loss: 0.0150 - acc: 0.999 - ETA: 8:34 - loss: 0.0150 - acc: 0.999 - ETA: 8:32 - loss: 0.0150 - acc: 0.999 - ETA: 8:31 - loss: 0.0150 - acc: 0.999 - ETA: 8:29 - loss: 0.0150 - acc: 0.999 - ETA: 8:28 - loss: 0.0150 - acc: 0.999 - ETA: 8:26 - loss: 0.0150 - acc: 0.999 - ETA: 8:25 - loss: 0.0150 - acc: 0.999 - ETA: 8:24 - loss: 0.0150 - acc: 0.999 - ETA: 8:22 - loss: 0.0149 - acc: 0.999 - ETA: 8:21 - loss: 0.0149 - acc: 0.999

563/563 [==============================] - ETA: 3:50 - loss: 0.0140 - acc: 0.999 - ETA: 3:49 - loss: 0.0140 - acc: 0.999 - ETA: 3:47 - loss: 0.0140 - acc: 0.999 - ETA: 3:46 - loss: 0.0140 - acc: 0.999 - ETA: 3:44 - loss: 0.0140 - acc: 0.999 - ETA: 3:43 - loss: 0.0140 - acc: 0.999 - ETA: 3:41 - loss: 0.0140 - acc: 0.999 - ETA: 3:40 - loss: 0.0140 - acc: 0.999 - ETA: 3:38 - loss: 0.0140 - acc: 0.999 - ETA: 3:37 - loss: 0.0140 - acc: 0.999 - ETA: 3:35 - loss: 0.0140 - acc: 0.999 - ETA: 3:34 - loss: 0.0140 - acc: 0.999 - ETA: 3:32 - loss: 0.0140 - acc: 0.999 - ETA: 3:31 - loss: 0.0140 - acc: 0.999 - ETA: 3:29 - loss: 0.0140 - acc: 0.999 - ETA: 3:28 - loss: 0.0140 - acc: 0.999 - ETA: 3:26 - loss: 0.0140 - acc: 0.999 - ETA: 3:25 - loss: 0.0140 - acc: 0.999 - ETA: 3:23 - loss: 0.0140 - acc: 0.999 - ETA: 3:22 - loss: 0.0140 - acc: 0.999 - ETA: 3:20 - loss: 0.0140 - acc: 0.999 - ETA: 3:19 - loss: 0.0142 - acc: 0.999 - ETA: 3:17 - loss: 0.0142 - acc: 0.999 - ETA: 3:16 - loss: 0.0142 - acc: 0.999